In [1]:
import numpy as np
import theano
import theano.tensor as T

In [3]:
x = T.dscalar("x")
y = T.dscalar("y")

f = theano.function([x, y], x ** 2 + y ** 2)

f(0, 0)

array(0.0)

In [285]:
e = np.array([[0, 0, 0], [1, 1, 1], [1, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, -1, 1]], dtype=float)
s = 0.05
n_ = np.array([1, 1, 1])
n_ = n_ / np.sqrt((n_ ** n_).sum())
(n_ * n_).sum()

print e

[[ 0.  0.  0.]
 [ 1.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0. -1.  1.]]


In [312]:
event = theano.shared(e, "event")

theta = T.dvector("theta")
phi = T.dvector("phi")

nx = T.cos(theta)
ny = T.sin(theta) * T.sin(phi)
nz = T.sin(theta) * T.cos(phi)

n = T.stack(nx, ny, nz).T
sigma = theano.shared(0.05, 'sigma')

scalar = T.dot(event, n.T)
project = event - T.tensordot(scalar, n, axes=([], []))
delta_sq = T.sum(project * project, axis=1)
r = T.sum(T.exp(-delta_sq / sigma))

#rr_hess = theano.gradient.hessian(r, [theta, phi])

rr = theano.function([phi, theta], project)

In [313]:
rr([0.0, 0.1], [0.0, 0.1])

ValueError: Input dimension mis-match. (input[0].shape[2] = 7, input[1].shape[2] = 2)
Apply node that caused the error: Elemwise{Sub}[(0, 1)](InplaceDimShuffle{x,x,0,1}.0, Reshape{4}.0)
Inputs types: [TensorType(float64, (True, True, False, False)), TensorType(float64, 4D)]
Inputs shapes: [(1, 1, 7, 3), (7, 2, 2, 3)]
Inputs strides: [(168, 168, 24, 8), (96, 48, 24, 8)]
Inputs values: ['not shown', 'not shown']

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [255]:
s(1, 2)

array([[ 1.,  2.]])

In [190]:
event.set_value(np.ones((3,2)))

In [191]:
event.get_value()

array([[ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.]])

In [217]:
x = T.dmatrix("x")
a = theano.shared(1.0, "a")

y = T.sum((x - a) ** 2)

dy = theano.function([x], theano.gradient.hessian(y, x))

AssertionError: tensor.hessian expects a (list of) 1 dimensional variable as `wrt`

In [215]:
from theano import pp

In [216]:
pp(dy.maker.fgraph.outputs[0])

'for{cpu,scan_fn}(Shape_i{0}(x), Constant{1}[int64:int64:int8], Shape_i{0}(x), Elemwise{Composite{(i0 * (i1 - i2))}}(TensorConstant{(1,) of 2.0}, x, a), alloc(TensorConstant{(1,) of 0.0}, Shape_i{0}(x)), alloc(TensorConstant{1.0}, Shape_i{0}(x)), alloc(TensorConstant{(1,) of 2.0}, Shape_i{0}(x)))'